In [ ]:
# To Do:

### Make a filterless set
Use the script to make a smaller parallel set out of the same issue ids without the field filters, and see if it does better with a small-enough-not-to-be-a-pain amount of data. Load all the volumes to the preset lists.

### Export better, for subsetting
Figure out how to re-json the data (I think it's 'dump' or 'dumps') and then you don't need the smaller dataset, and can maybe do cleaner analysis

### Find a better ML library?
Something that lets me treat numbers AND text as features

In [1]:
import numpy
import urllib2
import json
import time

numpy.random.seed(0)
spider_ids = [int(line.rstrip('\n')) for line in open('spider_issue_ids.txt')]
random_spiders = numpy.random.choice(spider_ids,2000)

i_black = []
i_white = []

v_white = []
v_black = []

collections = []
originals = []

def load_file(filename,t):
    f = open(filename,'rb')
    lines = f.read().splitlines()
    target = []
    if t == "int":
        for x in lines:
            target.append(int(x))
    else:
        for x in lines:
            target.append(x)
    return target

resource_hash = {'volume':4050,'issue':4000}

key = "aff8790cd32512f45b429bb78cc21a7a87cf4d48"
url_base = "http://comicvine.gamespot.com/api/"
v_fields = ["name","id","description","count_of_issues","start_year","publisher","site_detail_url"]

In [2]:
#preset_b = [27474,18946,22645,22769,23235,23306,23402,26570,32701,34138,48343,23632,
#              25702,1283,2293,3490]
#preset_w = [85928,2127,73420,22146,22405,22719,18354,75557,85312,78355,2576,2863,6011,18024,
#           7515,5561,2686,2870,3012,11069,3519,28576,4421,9142,5048,10760,11306,5562,
#           5788,18177,9375,7255,11066,11497,33777,17998,18459,2128,2045,43884,6218]

preset_b = load_file("v_black","int")
preset_w = load_file("v_white","int")

collected_names = ["mcfarlane","stern","marvel masterworks", "marvel-masterworks", "omnibus", 
                   "essential", "epic", "collection","collected","trade paperback",
                   "hard cover","hardcover","tp/hc", "complete", "soft cover", "softcover"]

#resampling -- 2000 ids gave us 15 collections and 35 originals
numpy.random.seed(0)
random_spiders = numpy.random.choice(spider_ids,10000)


In [7]:
def construct_url(i,resource_str,fields=[]):
    resource_n = resource_hash[resource_str]
    url = url_base+resource_str+"/"+str(resource_n)+"-"+str(i)+"/?format=json&api_key="+key
#    print url
    if fields !=[]:
        url = add_fields(url,fields)
    return url

def add_fields(base,f=[]):
    url = base + "&field_list="
    j = 0
    for item in f:
        url = url+str(item)
        if j < len(f)-1:
            url = url+","
        j = j+1
    return url
                
def url_to_data(url):
    req = urllib2.Request(url)
    req.add_header('User-agent', 'Mozilla 5.10')
    res = urllib2.urlopen(req)
    data = json.load(res)
    return data

def check_marvel(v):
    pub = v['publisher']
    if pub is None:
        return False
    if pub['id'] == 31:
        return True
    else:
        return False
    
def blacklist(idat, vdat):
    i_black.append(idat['id'])
    v_black.append(vdat['id'])
    collections.append(vdat)
    
def whitelist(idat, vdat):
    i_white.append(idat['id'])
    v_white.append(vdat['id'])
    originals.append(vdat)

def keep_track_of_length(ls):
    if len(ls)%10 == 0:
        if ls == collections:
            print("collections")
        if ls == originals:
            print("originals")
        print(len(ls))
    
def test_id(i):
    if (i in i_black) or (i in i_white):
        print("known issue")
        return
    issue_data = url_to_data(construct_url(i,'issue'))['results']
#    print(issue_data)
    title = issue_data['volume']['name'].lower()
    vi = issue_data['volume']['id']
    time.sleep(-time.time()%1)
    if vi in v_white:
        i_white.append(i)
        print("known volume")
        return
    if vi in v_black:
        i_black.append(i)
        print("known volume")
        return
    vol_data = url_to_data(construct_url(vi,'volume'))['results']#,v_fields))['results']
    if (check_presets(issue_data,vol_data)):
        print("preset volume")
        return
    if (not check_marvel(vol_data)):        
        print("not marvel")
        return
    for term in collected_names:
        if term in title:
            blacklist(issue_data, vol_data)
            print("keyword filter: "+title)
            return
    if(ask_if_collection(vol_data['site_detail_url'])):
#        print(str(issue_data['id'])+','+str(vol_data['id']))
        blacklist(issue_data, vol_data)
    else:
#        print(str(issue_data['id'])+','+str(vol_data['id']))
        whitelist(issue_data, vol_data)
    keep_track_of_length(collections)
    keep_track_of_length(originals)
    return
    
def test_id_list(ls=[]):
    j = 0
    for item in ls:
        time.sleep(-time.time()%1)
        test_id(item)
        if (len(ls)-j)%100==0:
            print str(len(ls)-j)+" ids left to go!"
            archive(v_black,"v_black",j)
            archive(v_white,"v_white",j)
            archive(i_black,"i_black",j)
            archive(i_white,"i_white",j)
        j=j+1
        
## This notebook keeps freezing up and I keep losing data, so...let's make exporting part of the game.        
def archive(ls,filen,num):
    new_file = filen+str(num)
    write_file(ls,new_file)

In [4]:
## Function to fast-track whitelist the preset vols -- when they come up in the random crawl

def check_presets(idat,vdat):
    if vdat['id'] in preset_w:
        whitelist(idat,vdat)
        return True
    if vdat['id'] in preset_b:
        blacklist(idat,vdat)
        return True
    return False

In [14]:
write_json_file(collections,"collections")
write_json_file(originals,"originals")

In [12]:
def write_file(ls,filename):
    f = open(filename, 'w')
    if type
    for item in ls:
        f.write("%s\n" % item)

def convert_to_star(ls):
    arr = []
    for item in ls:
        json_string = json.dumps(item)
        arr.append(json_string)
    return arr
        
def write_json_file(ls,filename):
    f = open(filename,'w')
    json_string_arr = convert_to_star(ls)
    for item in json_string_arr:
        f.write("%s\n" % item)
#write_file(collections,"collections")
#write_file(originals,"originals")
#write_file(i_black,"i_black")
#write_file(i_white,"i_white")
#write_file(v_black,"v_black")
#write_file(v_white,"v_white")

In [ ]:
for url in urls:
    data = url_to_data(url)
    json_string = json.dumps(data)
    print "json string = "+json_string[0:10]
    json_arr.append(json_string)
    

testf = open('testfile','w')
for item in json_arr:
    print "item = "+item[0:10]
    testf.write("%s\n" % item)
    
testf2 = load_file('testfile','r')
print testf2

In [11]:
#print preset_b[0:5]
#print preset_w[0:5]

test_id_list(random_spiders[310:2000])

not marvel
not marvel
not marvel
known issue
known issue
not marvel
known issue
known issue
not marvel
not marvel
known issue
not marvel
known issue
not marvel
known issue
known issue
known issue
known issue
known issue
not marvel
known issue
not marvel
known issue
known issue
known issue
not marvel
known issue
known issue
known issue
known issue
known issue
known issue
known issue
known issue
not marvel
not marvel
known issue
known issue
known issue
known issue
known issue
not marvel
known issue
not marvel
not marvel
known issue
known issue
known issue
known issue
not marvel
known issue
known issue
known issue
not marvel
known issue
known issue
known issue
known issue
known issue
not marvel
not marvel
known issue
known volume
preset volume
preset volume
preset volume
known volume
not marvel
not marvel
not marvel
not marvel
not marvel
preset volume
not marvel
preset volume
preset volume
known volume
not marvel
not marvel
not marvel
not marvel
known volume
not marvel
preset volume
prese

KeyboardInterrupt: 

In [37]:
print len(collections)

125


In [6]:
def ask_if_collection(link):
    print link
    while True:
        answer = raw_input()
        if answer == 'c':
            return True
        if answer == 'o':
            return False
        print("Enter 'c' for collection, 'o' for original:")
    